#   _Key Derivation Function_ 
## LBK, Kollbi (Insert names later)
###   Podstawowe informacje
1.  Funkcje KDF to algorytmy kryptograficzne zwracające sekret lub wiele sekretów z tajnej wartości.    
        Np. Stworzenie klucza prywatnego (jeden sekret) z hasła użytkownika (tajna wartość).
1.  Metody te powstały ponieważ hasła zazwyczaj są proste i do przewidzenia. Pojawia się potrzeba stworzenia mocnego klucza.  
Jednak nikt nie będzie pamiętał 64 znakowego hasła. Dlatego z pomocą przychodzi `key stretching`. Czyli proces w którym tworzony jest bezpieczniejszy i (teoretycznie) odporniejszy na ataki brute-force nowy klucz.  
Dostając proste hasło zostają wykonane operacje, które dosłownie `rozciągają` podany ciąg znaków. Do key streching'u można zaliczyć proste funkcje hashujące jak SHA, MD5 lub szyfry blokowe.
```
                                               _             _  ______  _____                                   _       
     _ __   __ _ ___ _____      _____  _ __ __| |           | |/ /  _ \|  ___|           ___  ___  ___ _ __ ___| |_ ___  
    | '_ \ / _` / __/ __\ \ /\ / / _ \| '__/ _` |   =====\  | ' /| | | | |_     ====\   / __|/ _ \/ __| '__/ _ \ __/ __|
    | |_) | (_| \__ \__ \\ V  V / (_) | | | (_| |   =====/  | . \| |_| |  _|    ====/   \__ \  __/ (__| | |  __/ |_\__ \
    | .__/ \__,_|___/___/ \_/\_/ \___/|_|  \__,_|           |_|\_\____/|_|              |___/\___|\___|_|  \___|\__|___/
    |_|                                                     
```

### Funkcje KDF to nie tylko funkcje hashujące! Jednak mają wspólną część.
1. Tak samo jak funkcje haszujące szyfrują dane deterministycznie. (To samo wejście - to samo wyjście)
1. Na wyjściu jest zawsze ciąg o takiej samej długości. Bez znaczenia jak duże były dane wejściowe.
1. Działa tylko w jedną stronę. Nie odzyska wejścia danych z wyjścia.


### Lepsze zrozumienie od podstaw
Najprostszą funkcję KDF można przedstawić wykorzystując np. algorytm SHA256. Jednak jest to niebezpieczne. Proste hasze są podatne na ataki słownikowe (np. tęczowe tablice (Rainbow table))

function(`password`, `sha256`) --> `key`

Pierwszym krokiem w stronę KDF (i zabezpieczeniem) jest dodanie `soli` (ang. `salt`).  
`Salt` jest wartością przechowywaną razem z uzyskanym kluczem. Używana, żeby uniemożliwić atak brute-force.  
W skrócie: `Dodanie soli == Odporność na Tęczowe Tablice`

function_with_salt(`salt`, `password`, `sha256`) --> `key`

Funkcję wyżej można opisać jako `HKDF` (`HMAC-based key derivation function`). Ciągle jest mniej bezpieczna niż aktualniejsze KDF. Zaleca się korzystanie z PBKDF2, Bcrypt, Scrypt (ten projekt) oraz Argon2.


### Przykład 1.0
Wykorzystanie `HMAC` (Hash-based message authentication code) z solą, hasłem użytkownika oraz funkcją SHA256 do pozyskania klucza.
1. Sprawdź co się stanie jak zmienisz sól. Może to być bardzo mała zmiana. Porównaj parę wyników dla tego samego hasła. 
1. Czy klucze z nową solą znacznie różnią się od głównego? Jeśli tak to dlaczego. ([Lawinowość](https://en.wikipedia.org/wiki/Avalanche_effect))
1. Porównaj dwa klucze z tym samym hasłem w następujący sposób:  
    1. Dla samej funkcji haszującej `sha256(b"password")`
    1. Następnie stwórz dwie różne sole.
    1. Porównaj klucze dołączając sól po haśle `sha256(b"password+salt")`

In [44]:
from hmac import HMAC
from hashlib import sha256

salt = b'Crypto'
password = b'IsFun'

key = HMAC(salt, password, digestmod=sha256)

print(key.digest().hex())  # Desired output: `a773d6dd2e141ddc22a578716a5da01e695286e03abf3b09abb3c48c296e1a70`

a773d6dd2e141ddc22a578716a5da01e695286e03abf3b09abb3c48c296e1a70


### Zasosowanie `soli`
Wielokrotne użycie soli dla tego samego hasła umożliwia pozyskanie wielu innych kluczy. Jest to często stosowane rozwiązanie.  
Dla przykładu: można posiadać w bazie dwóch użytkowników z takim samym hasłem. Zwykłe zahaszowanie hasła w obu przypadkach zwróci to samo.  
Jednak wygenerowanie dla każdego z nich unikalnej wartości (`salt`) i dołączenie do hasła, sprawia że otrzymane hasze są diametralnie różne. ([Lawinowość](https://en.wikipedia.org/wiki/Avalanche_effect))  
KDF pomimo znajomości klucza z `salt1` nie jest wstanie znaleźć klucza z `salt2`.

### Po podstawach

Wiedząc co rozróżnia funkcje haszujące, KDF oraz czym jest sól zwrócona zostanie uwaga na to co powinna zapewniać dobra funkcja KDF.

1. Memory Hardness #TODO <--- Wydaje się super ważne ale nie miałem głowy, żeby już tego szukać...  